In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sqlite3    
import re   
import requests   
from bs4 import BeautifulSoup   
import plotly.graph_objects as go
#import win32com.client as win32
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_rows', None)

#### Funções Data

In [3]:
def day_n (offset):         
    
    return datetime.today()-timedelta(days = offset)         

In [4]:
def sunday(offset, day):    
    
    if day.weekday()==6:     
        day = day_n(offset+2)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)         
    
    return day   

####  Caso seja domingo essa função retorna o última dia antes de começar o final de semana, seja ele feriado ou não.  ####  Caso não seja domingo ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [5]:
def saturday(offset, day):
    
    if day.weekday()==5:     
        day = day_n(offset+1)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)     
    return day   

####  Caso seja sabado essa função retorna o última dia antes de começar o final de semana, 
####  seja ele feriado ou não.  
####  Caso não seja sábado ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [6]:
def holiday(offset, day, dataf):     
    
    for i in dataf:     
        if (i[8:]+"-"+i[5:7]+"-"+i[0:4])==day.strftime("%d-%m-%Y"):             
            return holiday(offset+1, day_n(offset+1), dataf)         
    
            
    if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):         
    
        
        if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):             
            
            return day         
        else:             
            return saturday(offset, day)         
    else:         
        return sunday(offset, day)         

In [7]:
##### Descobrindo o D-1 #####

def find_last_workday( offset, day, dataf): 
    
    if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
        
        if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):             
            
            if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):                 
                return day                 
                
            else:                     
                return saturday(offset, day)         
        else:                 
            return sunday(offset, day)         
    else:         
        return holiday(offset, day, dataf)          

##### Vetor de feriados

In [8]:
def holidays():
    
    feriados = pd.read_csv(r"C:\Users\psilv\Python\feriados.csv")   
 
 
    data = feriados['data'].tolist()   
 
    ########### Cria vetor de feriados 
    
    dataf=[] 
    
    for i in data:   
        
        dataf.append(i.strip()[-4:] + "-" + ("0"+ i.strip()[0] if i.find("/")==1 else i.strip()[0:i.find("/")]) + "-" + ("0" + re.search("/(.*?)/", i.strip()).group(1) if len(re.search("/(.*?)/", i.strip()).group(1))==1 else re.search("/(.*?)/", i.strip()).group(1)))
        
    return dataf     

In [9]:
def past_n_days(n):     
    day = datetime.today()-timedelta(days = n)     
    print("O último dia corrido é: " +  day.strftime("%d-%m-%Y"))     
    return day   

#####  Função auxiliar para criar Vetor de dias úteis 

In [10]:
##### Date Format input  YY-MM-dd,  run_days são números inteiros

##### O input de data dentro da função pandas.date_range deve ser no formato YY-MM-dd

def date_array (last_day, run_days):

   
    Datas = pd.date_range(end = last_day, periods = run_days).to_pydatetime().tolist()

    
    datas=[] 

    
    for k in range(len(Datas)):

        datas.append(Datas[len(Datas) - k -1])



 
    return datas

#### Função iterativa para criar Vetor de dias úteis.

In [11]:
#### Função para obter um vetor de dias úteis de um tamanho específico, por ser iterativa é custosa computacionalmente

#### Date format input dd-MM-YY, length é número inteiro


def work_days_array_length(last_day, length):


    i=length

    while len(date_array(last_day, i))!= length:

        if len(date_array(last_day, i))==length:

            break

        i+=1

    return date_array(last_day, i)

In [12]:
####### Criando vetor desde o ultimo dia do banco até o D-1

####### Start é a mais antiga, e End a mais recente

####### Date Format input  YY-MM-dd (SQL)


def  work_days_array_btw_dates(start_date, end_date):
    

    end = date(int(end_date[:4]), int(end_date[5:7]), int(end_date[-2:]))

    start = date(int(start_date[:4]), int(start_date[5:7]), int(start_date[-2:]))

    
    dif = end-start
    

    dif = int(abs(dif.days))
    
    
    print("Números de dias a serem atualizados:  " + str(dif))
    print("\n")
    
    #### No caso start_date=end_date é preciso desse if 
    
    if dif == 0:
        dif=2
            


    dates = date_array(end_date, 2*dif)

    aux=0
    

    for i in dates:

        if i.strftime("%Y-%m-%d")==start_date:

            aux = dates.index(i)
                 
    
    return dates[:aux+1]

#### Função para deletar linha específica

In [13]:
def delete_row_sql(table_name, column_name, conn, desired_info):
    
    
    query = ("""DELETE""" + 
             """ FROM """ + table_name +
             """ WHERE """ + column_name +
             """ = '""" + desired_info +
             """'; """)
    
    conn.execute(query)
    
    print("Deleted as Follows:")
    print("\n")
    print(query)

#### Função para obter um dataframe a partir de uma query do banco de dados

In [14]:
### Criar função para obter um dataframe entre datas específicas a partir de uma query em SQL

### MELHORAR PRA RANGE DE DATAS


def df_from_sql_query(table_name, date_column_name, conn, start_date, end_date):
    
    
    query = ("""SELECT * """ + 
             """ FROM """ + table_name +
             """ WHERE """ + date_column_name +
             """ BETWEEN strftime('%Y-%m-%d %H:%M:%S', '""" + start_date +
             """') AND strftime('%Y-%m-%d %H:%M:%S', '""" + end_date +
             """') ORDER BY """ + date_column_name + """; """)
    
    #print(query)

    df = pd.read_sql_query(query, conn)
    
    
    return df   

In [15]:
# df_from_sql_query('bitcoin', "Date", conn, "2022-01-28", "2022-07-25")

#### Inserindo novas linhas no banco de dados

In [16]:
##### Tornar essa função genérica para qualquer dataframe

def insert_n_lines_database(cursor, df, table_name, date_column_name):
    
           
    print("\n")     
    print("Os dados antes da atualização eram:")   
    print("\n")  
    
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)             
 
    print("\n")     
   
    insert_n_rows(cursor, df, table_name)             
       
    print("\n")   
    print("Os novos dados são:")   
    print("\n")     
                   
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)         
    print("\n")             
            

#### Atualizando as bases de dados

In [17]:
def updating_database(cursor, date_column_name):      
    
    for i in assets:
        
        print('\n')
        
        day = past_n_days(1)
    
        day = day.strftime("%Y-%m-%d")
        
        ###### Verificando a última data atualizada no banco de dados   
        last_date_db = get_last_db_date(cursor, date_column_name, i)[:10]        

        if last_date_db == day:         
            print("Os dados da cripto " + str(i) + " estão Atualizados!!")         
        else:         
            print("\n")     
            print("O último dia atualizado no banco de dados é: " + last_date_db)   
            print("\n")         

            #### Vetor de datas   
            dates = work_days_array_btw_dates(last_date_db, day)[:-1]   
            print("Datas a serem atualizadas no banco de dados:")         
            print("\n")         

            for j in range(len(dates)):             
                print(dates[j])             


            print("\n")
            print("Downloading from Yahoo Finance API ....")         
            print("\n")         

            #### Dataframe com os resultados
            
            print("Moeda Atualizada: " + assets[i])
    

            df = get_data_btw_date(assets[i],dates[-1:][0].strftime("%Y-%m-%d"), dates[0].strftime("%Y-%m-%d"))     


            #### Inserindo novos dados           
            insert_n_lines_database(cursor, df, i, date_column_name)  

            ##### Adicionar o insert lines dentro da função Web Scrap para atualizar o banco de dados em cada iteração e não
            ##### correr o risco de falhar o web scraping e perder os dados
    
    

#### Função para captar os dados da API do Yahoo Finance

In [53]:
#### Nome da moeda, exemplo: Ethereum é ETH-USD
#### Data format YY-MM-DD

def get_data_btw_date(crypto_name, start_date, end_date):
    
    crypto_df = yf.download(crypto_name, start= start_date, end= end_date)
    
    crypto_df = crypto_df.reset_index(level=0)
    
    print(crypto_df.dtypes)
    print(crypto_df)
    
    #crypto_df['Date'] = pd.to_datetime(crypto_df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    #crypto_df['Date'] = crypto_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    #print(crypto_df.dtypes)
    #print(crypto_df)

    return crypto_df

In [19]:
a = get_data_btw_date('ADA-USD', '2022-08-21', '2022-08-21')

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
        Date      Open      High       Low     Close  Adj Close     Volume
0 2022-08-21  0.451266  0.472278  0.448482  0.463546   0.463546  616272980
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
        Date      Open      High       Low     Close  Adj Close     Volume
0 2022-08-21  0.451266  0.472278  0.448482  0.463546   0.463546  616272980


In [ ]:
print(a.dtypes)

#### Funções SQL

In [ ]:
## Insert crypto name in lower case

In [20]:
#### index can be False or True, to me worked as False

def create_table(df, name_desired, conn, index):
    
        df.to_sql(name_desired, conn, if_exists='fail', index=index)   
    


In [21]:
def rename_table(table_name, new_table_name, cursor):
    
    try:
        
        for i in get_names_tables(cursor):
        
            if new_table_name == i[0]:
                
                print("O Nome já está em uso!")
            
                raise Exception

        cursor.execute("""ALTER TABLE """ +  table_name + """ RENAME TO """ + new_table_name + """ ;""")

    except Exception:

        print("Não foi possível alterar o nome da tabela!")
        

In [22]:
def show_table(table_name, cursor):     
    
    c=cursor.execute("""SELECT * FROM """ + table_name ).fetchall()     
    for i in c:      
        print(i)     
    return c     

In [23]:
def delete_table(name, cursor):   
    
    cursor.execute("""DROP TABLE """ +  name ).fetchall()   

In [24]:
def last_database_day(crypto_name):

    sql_str = '''SELECT Date FROM ''' + crypto_name + ''' ORDER BY Date DESC LIMIT 1;'''

    c.execute(sql_str)

    date = c.fetchone()

    return date[0][:10]
    

In [25]:
def get_names_tables(cursor):     
    
    cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')   
    
    return(cursor.fetchall())

In [26]:
def count_rows_table(cursor, table_name): 
    
    c = cursor.execute("""SELECT COUNT(*) FROM """ + table_name).fetchall()     
    return c[0][0]   

In [27]:
def get_columns_names(cursor, table_name): 
    
    c = cursor.execute("""PRAGMA table_info(""" + table_name + """);  """ ).fetchall()     
    col_names = []     
    for i in c:         
            col_names.append(i[1])
            
    return col_names   

In [28]:
def get_last_n_rows(cursor, table_name, n_rows, column_name):
    
    c = cursor.execute("""SELECT * FROM """ 
                       + table_name + 
                       """ ORDER BY """ +column_name + 
                       """ DESC LIMIT """+ str(n_rows) + 
                       """ ; """  ).fetchall()     
#     for i in c:      
#         print(i)


    
    return c     

In [29]:
def insert_n_rows(cursor, df, table_name):     
 
   
    new_values = []   
    
    for i in range(len(df)):    
        
        new_values.append(tuple(df.iloc[i]))         
    
    s=""     
    
    for i in new_values:   
        
        s+=str(i) + ","   
        
    
    s=s[:-1] 

    #print(s)
    
    
    cursor.execute("""INSERT INTO """ + table_name + """ VALUES """ + s + """ ; """  )     
    

In [30]:
def distinct_values(table_name, column_name, cursor):
    
    c = cursor.execute("""SELECT """ + column_name + """, COUNT(*) """ + """ FROM """ + table_name + """ GROUP BY """ +  column_name + """ ; """  ).fetchall()     
    
    
    for i in c:
        
        print(i)
    
       

In [31]:
def change_columns_names(cursor, table_name, column_old_name, column_new_name):   
 
    print("Old Columns Names: " + str(get_columns_names(cursor, table_name)) )     
    c = cursor.execute("""ALTER TABLE """ + table_name +                                 """ RENAME COLUMN """ + column_old_name +                                 """ TO """+column_new_name+                                 """ ; """  ).fetchall()     
    print("New Columns Names: " + str(get_columns_names(cursor, table_name)) )     
 

In [32]:
def get_last_db_date(cursor,column_name, table_name):  
    
    c=cursor.execute("""SELECT """ + column_name + 
                     """ FROM """ + table_name + 
                     """ ORDER BY """ + column_name + 
                     """ DESC LIMIT 1 ; """).fetchall()     
    return c[0][0]   

In [33]:
def database_info(cursor, table_name, date_column_name):
    
        
    
    print("O banco de dados contém as seguintes tabelas: ")
    
    print("\n")
    
    for i in get_names_tables(cursor):
        print(i[0])
    
    print("\n")
    
    print("O nome da tabela solicitada é :" + table_name)
    
    print("\n")
    
    print("O nome das colunas são: ")
    
    print("\n")
    
    print(get_columns_names(cursor, table_name))
    
    print("\n")
    
    print("O número total de linhas é: " + str(count_rows_table(cursor, table_name)))
    
    print("\n")
    
    print("A data mais recente dos dados é: " + str(get_last_db_date(cursor,date_column_name, table_name)))
    
    print("\n")
    
    print("Todas as datas distintas e quantas linhas cada uma possui são: ")
    
    print("\n")
    
    print(distinct_values(table_name, date_column_name, cursor)) 
    
    

In [34]:
def get_new_crypto_in_yf(crypto_name, table_title, conn, end_date, index=False):
    
 
    df = get_data_btw_date(crypto_name, '2000-01-01', end_date)
        
    #### Data escolhida é bem antes do surgimento das criptomoedas no planeta terra
        
        
    create_table(df, table_title, conn, index)
    

In [35]:
###### updating entire database for all assets ##########

def insert_new_criptos_in_db(assets):
    
    day = past_n_days(1)
    
    day = day.strftime("%Y-%m-%d")

    for i in assets:
        
        try:

            get_new_crypto_in_yf(assets[i], i, conn, day, index=False)
            
        
        except Exception:
            
            print(str(assets[i]+ " já  está no banco de dados!"))
            
            continue

In [36]:
assets = { 'BTC': 'BTC-USD', 
           'ETH': 'ETH-USD', 
           'USDT':'USDT-USD', 
           'USDC':'USDC-USD', 
           'BNB':'BNB-USD',
           'BUSD':'BUSD-USD',
           'XRP': 'XRP-USD',
           'ADA': 'ADA-USD',
           'SOL': 'SOL-USD',
           'DOGE':'DOGE-USD',
           'DOT': 'DOT-USD',
           'HEX': 'HEX-USD',
           'SHIB': 'SHIB-USD',
           'DAI': 'DAI-USD',
           'WTRX': 'WTRX-USD',
           'AVAX':'AVAX-USD',
           'MATIC': 'MATIC-USD',
           'TRX': 'TRX-USD',
           'STETH': 'STETH-USD',
           'WBTC': 'WBTC-USD',
           'UNI1': 'UNI1-USD',
           'LEO': 'LEO-USD',
           'ETC': 'ETC-USD',
           'LTC': 'LTC-USD'
         }

#### Rodando

In [40]:
conn = sqlite3.connect('crypto_data')
c = conn.cursor()

In [43]:
#updating_database(c, "Date")

In [44]:
#database_info(c, 'BTC','Date')

In [54]:
delete_table('BTC', c)

In [55]:
database_info(c, 'BTC','Date')

O banco de dados contém as seguintes tabelas: 


ETH
DOT
ADA
USDT
USDC
BNB
BUSD
XRP
SOL
DOGE
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
UNI1
LEO
ETC
LTC


O nome da tabela solicitada é :BTC


O nome das colunas são: 


[]




OperationalError: no such table: BTC

In [56]:
get_new_crypto_in_yf("BTC-USD", "BTC", conn, "2023-06-06", index=False)

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
           Date          Open          High           Low         Close  \
0    2014-09-17    465.864014    468.174011    452.421997    457.334015   
1    2014-09-18    456.859985    456.859985    413.104004    424.440002   
2    2014-09-19    424.102997    427.834991    384.532013    394.795990   
3    2014-09-20    394.673004    423.295990    389.882996    408.903992   
4    2014-09-21    408.084991    412.425995    393.181000    398.821014   
5    2014-09-22    399.100006    406.915985    397.130005    402.152008   
6    2014-09-23    402.092010    441.557007    396.196991    435.790985   
7    2014-09-24    435.751007    436.112000    421.131989    423.204987   
8    2014-09-25    423.156006    423.519989   

In [57]:
database_info(c, 'BTC','Date')

O banco de dados contém as seguintes tabelas: 


ETH
DOT
ADA
USDT
USDC
BNB
BUSD
XRP
SOL
DOGE
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
UNI1
LEO
ETC
LTC
BTC


O nome da tabela solicitada é :BTC


O nome das colunas são: 


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


O número total de linhas é: 3185


A data mais recente dos dados é: 2023-06-06 00:00:00


Todas as datas distintas e quantas linhas cada uma possui são: 


('2014-09-17 00:00:00', 1)
('2014-09-18 00:00:00', 1)
('2014-09-19 00:00:00', 1)
('2014-09-20 00:00:00', 1)
('2014-09-21 00:00:00', 1)
('2014-09-22 00:00:00', 1)
('2014-09-23 00:00:00', 1)
('2014-09-24 00:00:00', 1)
('2014-09-25 00:00:00', 1)
('2014-09-26 00:00:00', 1)
('2014-09-27 00:00:00', 1)
('2014-09-28 00:00:00', 1)
('2014-09-29 00:00:00', 1)
('2014-09-30 00:00:00', 1)
('2014-10-01 00:00:00', 1)
('2014-10-02 00:00:00', 1)
('2014-10-03 00:00:00', 1)
('2014-10-04 00:00:00', 1)
('2014-10-05 00:00:00', 1)
('2014-10-06 00:00:00', 1)
('2014-10-07 0

In [ ]:
def plot(title,crypto, conn, startDate, endDate):
    
    
    df=df_from_sql_query(crypto, "Date", conn, startDate, endDate)
    
    df['Date']= pd.to_datetime(df['Date'])
    
    df=df.drop(columns=['Adj Close'])
    
#     eth_df.set_index('Date')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.update_layout(
        title= title,
        yaxis_title='Price',

    )

    fig.write_html(title + ".html")
    fig.write_html(title + ".html")

    fig.show()

In [ ]:
plot('ETHER-USD','ETH', conn, '2022-08-01', '2022-09-14')

In [ ]:
plot('BTC-USD','BTC', conn, '2022-08-01', '2022-09-14')